## Import packages

In [ ]:
from matplotlib import gridspec
from matplotlib.collections import PolyCollection
from matplotlib.path import Path
from matplotlib.ticker import LogLocator, ScalarFormatter
import numpy as np
import matplotlib.pyplot as plt
from ortools.math_opt.python import mathopt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from cpwllib.implementation import MILP_or_QP_variables_and_constraints, N_from_target_error, list_faces_from_N, list_faces_from_N_DC, \
    plot_faces_3d, equations_from_faces_3d, evaluate_z_from_equations, \
    equations_sum_convex

In [ ]:

target_error = 0.001

N = N_from_target_error(target_error)
# get triangles faces
faces = list_faces_from_N(N, method='triangles')
plot_faces_3d(faces)
# get polygon faces
faces = list_faces_from_N(N, method='polygons')
plot_faces_3d(faces)

# produce a random (x,y) number
x_fix, y_fix = np.random.rand(2)
print(f"x: {x_fix:.4f}, y: {y_fix:.4f},")
print(f"Real z = x*y: {x_fix*y_fix:.4f}")

faces = list_faces_from_N(N, method='triangles')
list_coeffs, list_equations = equations_from_faces_3d(faces)
z_values = evaluate_z_from_equations(np.array([[x_fix, y_fix]]), 
                                     list_coeffs, list_equations)
print(f"Approximated z from triangles: {z_values[0]:.4f}")

faces = list_faces_from_N(N, method='polygons')
list_coeffs, list_equations = equations_from_faces_3d(faces)
z_values = evaluate_z_from_equations(np.array([[x_fix, y_fix]]), 
                                     list_coeffs, list_equations)
print(f"Approximated z from polygons: {z_values[0]:.4f}")

list_coeffs_j, list_equations_j, list_coeffs_k, list_equations_k = equations_sum_convex(N)
z_values_j = evaluate_z_from_equations(np.array([[x_fix, y_fix]]), 
                                     list_coeffs_j, list_equations_j)
z_values_k = evaluate_z_from_equations(np.array([[x_fix, y_fix]]), 
                                     list_coeffs_k, list_equations_k)
print(f"Approximated z from sum of convex: {(z_values_j+z_values_k)[0]:.4f}")

In [ ]:
# produce random values between 30 and 50 in a 5-by-10 matrix
N_nodes = 20
N_time = 10
X_Y_UB = 50.
X_Y_LB = 30.

x_fix, y_fix = X_Y_LB + (X_Y_UB-X_Y_LB)*np.random.rand(2,N_nodes,N_time)

# create a dummy model
model = mathopt.Model(name='Test')
# dictionary of variables mimicking bypass tool's T and Q
variables = {}
variables['T_gg'] = [[model.add_variable(lb=30., ub=50., name=f'T_gauge_{j}_{t}') 
                      for t in range(N_time)] for j in range(N_nodes)]
variables['q_gg'] = [[model.add_variable(lb=30., ub=50., name=f'q_gg_{j}_{t}') 
                      for t in range(N_time)] for j in range(N_nodes)]

# MOST IMPORTANT FUNCTION
# This add necessary variables to the model
new_variables = MILP_or_QP_variables_and_constraints(model, variables['T_gg'], variables['q_gg'],
                                   target_error=target_error, 
                                   partition_method='sum of convex',
                                   logarithmic_encoding=True)

# dummy objective function
model.maximize(new_variables["Z"][0][0])

# now fix T and q to force the solution
for p in range(N_nodes):
    for t in range(N_time):
        variables['T_gg'][p][t].lower_bound = x_fix[p][t]
        variables['T_gg'][p][t].upper_bound = x_fix[p][t]
        variables['q_gg'][p][t].lower_bound = y_fix[p][t]
        variables['q_gg'][p][t].upper_bound = y_fix[p][t]
   
# solve the problem
params = mathopt.SolveParameters(enable_output=True)
result = mathopt.solve(model, solver_type=mathopt.SolverType.GSCIP, params=params)
result.termination.reason
result.solve_stats

# Real Z values
real_Z_values = x_fix*y_fix

# Calculated by the model
model_Z_values = np.array([[result.variable_values()[new_variables["Z"][p][t]]
                  for t in range(N_time)] for p in range(N_nodes)])

Difference = real_Z_values-model_Z_values

print(f"Average error: theory = {(X_Y_UB-X_Y_LB)**2/(48*N**2):.3f}, actual = {abs(Difference).mean():.3f}")
print(f"Max error: theory = {(X_Y_UB-X_Y_LB)**2/(16*N**2):.3f}, actual = {abs(Difference).max():.3f}")


In [ ]:
#%% Try equations from Maris
def g_plus(points,N):
    x = points[:,0]
    y = points[:,1]
    return np.max(
        np.array(
            [(2*j+1)*x/(4*N) + (2*j+1)*y/(4*N) - (j*(j+1))/(4*N**2) for j in range(2*N)]
            ),
        axis=0)

def g_minus(points,N):
    x = points[:,0]
    y = points[:,1]
    return np.max(
        np.array(
            [-(2*k+1)*x/(4*N) + (2*k+1)*y/(4*N) - (k*(k+1))/(4*N**2) for k in range(-N,N)]
            ),
        axis=0)

# 1D points between 0 and 1
x = np.linspace(0, 1, 101)
y = np.linspace(0, 1, 101)

# create a 2D grid
X, Y = np.meshgrid(x, y)

# stack into coordinate pairs if needed
points = np.column_stack([X.ravel(), Y.ravel()])

Z = (g_plus(points,N) - g_minus(points,N)).reshape(-1,len(x))


fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111, projection="3d")
surf = ax.plot_surface(X, Y, Z, cmap="viridis", edgecolor="none")

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("f(x,y)")
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()


In [ ]:
#%% test quadratic

# create a dummy model
model = mathopt.Model(name='Test')

x = model.add_variable(lb=-1., ub=1., name="x")
y = model.add_variable(lb=-1., ub=1., name="y")
z = model.add_variable(lb=0., ub=1., name="z")
# a = model.add_binary_variable(name="a")
# a = model.add_variable(lb=0., ub=1., name="a")

# model.add_quadratic_constraint((0.0 <= x*x - y*y - z) <= 0.0)
model.add_quadratic_constraint((0.0 <= x*y - z) <= 0.0)
model.add_linear_constraint(x + y == 1.)
# model.add_quadratic_constraint((0.0 <= z*x - a) <= 0.0)
# model.add_linear_constraint(x<=a)
                               
model.maximize(z)

# x.lower_bound = 0.5
# x.upper_bound = 0.5
# y.lower_bound = 0.5
# y.upper_bound = 0.5

params = mathopt.SolveParameters(enable_output=True)
result = mathopt.solve(model, solver_type=mathopt.SolverType.GSCIP, params=params)
result.termination.reason
result.solve_stats
result.variable_values()[x]
result.variable_values()[y]

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(10,5),dpi=150)
fig = plt.figure(figsize=(10,5),dpi=150, constrained_layout=True)
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)

ax=[]
ax.append(fig.add_subplot(gs[0]))
ax.append(fig.add_subplot(gs[1]))

ax[0].set_xlabel('x')
ax[0].set_ylabel('y')

# Parameters
x0, y0 = 0, 0
delta = 1


# Define diamond corners
diamond = np.array([
    [x0,         y0 + delta],
    [x0 + delta, y0        ],
    [x0,         y0 - delta],
    [x0 - delta, y0        ],
    [x0,         y0 + delta]
])


# Plot diamond
ax[0].plot(diamond[:, 0], diamond[:, 1], 'k-', lw=2, label='Diamond Square')

# Plot centroid
# ax.plot(x0, y0, 'ro', label='Centroid')
ax[0].text(x0+0.05, y0+0.05, f'$(x_0,y_0)$')

# Plot axes
for k in [-1,0,1]:
    ax[0].axhline(y=y0+k*delta, color='gray', linestyle='--', linewidth=1)
    ax[0].axvline(x=x0+k*delta, color='gray', linestyle='--', linewidth=1)

# Annotate axis lines
# ax.text(x0 + 0.05, y0 + delta + 0.1, r'$x = x_0$', color='gray')
# ax.text(x0 + delta + 0.1, y0 + 0.05, r'$y = y_0$', color='gray')

# Define lines (edges of the diamond)
lines = [
    ((x0 - delta, y0), (x0, y0 + delta)),
    ((x0, y0 + delta), (x0 + delta, y0)),
    ((x0 + delta, y0), (x0, y0 - delta)),
    ((x0, y0 - delta), (x0 - delta, y0))
]

offset = 0.3  # outward distance from center for label placement

for (x1, y1), (x2, y2) in lines:
    # Midpoint
    x_mid = (x1 + x2) / 2
    y_mid = (y1 + y2) / 2

    # Direction vector of the edge
    dx = x2 - x1
    dy = y2 - y1

    # Normal vector pointing outward from (x0, y0)
    nx = y_mid - y0
    ny = -(x_mid - x0)
    norm = np.hypot(nx, ny)
    nx /= norm
    ny /= norm

    # Offset position away from center
    x_text = x_mid #+ offset * nx
    y_text = y_mid + 0.05 #+ offset * ny

    # Compute slope and angle
    m = (y2 - y1) / (x2 - x1)
    angle = np.degrees(np.arctan2(dy, dx))

    # Ensure text is upright
    if angle > 90:
        angle -= 180
    elif angle < -90:
        angle += 180

    # Place label
    ax[0].text(
        x_text, y_text,
        rf"$y = y_0 {'-' if m<0 else '+'}x {'+' if m<0 else '-'}x_0 {'+' if (x_mid-x0)*m<0 else '-'}\delta$",
        fontsize=9,
        rotation=angle,
        rotation_mode='anchor',
        ha='center',
        va='bottom',
        bbox=dict(boxstyle='round,pad=0.5', fc='none', ec='none', alpha=0.7)
    )

# Symbolic tick labels
xticks = [x0 - delta, x0, x0 + delta]
yticks = [y0 - delta, y0, y0 + delta]
ax[0].set_xticks(xticks)
ax[0].set_yticks(yticks)
ax[0].set_xticklabels([r'$x_0 - \delta$', r'$x_0$', r'$x_0 + \delta$'])
ax[0].set_yticklabels([r'$y_0 - \delta$', r'$y_0$', r'$y_0 + \delta$'])

# Final formatting
ax[0].set_aspect('equal')
ratio_lim = 1.0
ax[0].set_xlim(x0 - ratio_lim * delta, x0 + ratio_lim * delta)
ax[0].set_ylim(y0 - ratio_lim * delta, y0 + ratio_lim * delta)
# ax.legend()
ax[0].set_title('$R_0$ domain')

# alternative plot for |E|

# Create a square grid
x = np.linspace(-1, 1, 1000)
y = np.linspace(-1, 1, 1000)
x, y = np.meshgrid(x, y)

z = x * y

mask_idx = (x + y <= 1) & (x + y >= -1) & (y - x <= 1) & (y - x >= -1)

z[~mask_idx] = np.nan


# Plot

c = ax[1].pcolormesh(x, y, abs(z), cmap='viridis', shading='auto')
ax[1].set_aspect('equal')
# Add colorbar
cb_ax = fig.add_subplot(gs[2])
cb = fig.colorbar(c, cax=cb_ax)

# cb = fig.colorbar(c, ax=ax[1], pad=0.1)

# Define diamond corners
diamond = np.array([
    [x0,         y0 + delta],
    [x0 + delta, y0        ],
    [x0,         y0 - delta],
    [x0 - delta, y0        ],
    [x0,         y0 + delta]
])


# Plot diamond
ax[1].plot(diamond[:, 0], diamond[:, 1], 0, 'black', linewidth=0.3)

# Custom x and y ticks
ax[1].set_xticks([x0 - delta, x0, x0 + delta])
ax[1].set_xticklabels([r'$x_0 - \delta$', r'$x_0$', r'$x_0 + \delta$'])

ax[1].set_yticks([y0 - delta, y0, y0 + delta])
ax[1].set_yticklabels([r'$y_0 - \delta$', r'$y_0$', r'$y_0 + \delta$'])

ax[1].set_xlabel('x')
# ax[1].set_ylabel('y')

# Set abstract tick labels based on \delta
cb.set_ticks([0, 0.25])
cb.set_ticklabels([r'$0$', r'$\delta^2/4$'])

ax[1].set_title(r'Value of $|E(x,y)|$')

# plt.tight_layout()
plt.show()

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(10,5),dpi=150)
fig = plt.figure(figsize=(10,5),dpi=150, constrained_layout=True)
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)

ax=[]
ax.append(fig.add_subplot(gs[0]))
ax.append(fig.add_subplot(gs[1]))

ax[0].set_xlabel('x')
ax[0].set_ylabel('y')

# Parameters
x0, y0 = 0, 0
delta = 1


# Define diamond corners
diamond = np.array([
    [x0,         y0 + delta],
    [x0 + delta, y0        ],
    [x0,         y0 - delta],
    [x0 - delta, y0        ],
    [x0,         y0 + delta]
])


# Plot diamond
ax[0].plot(diamond[:, 0], diamond[:, 1], 'k-', lw=2, label='Diamond Square')

# Plot centroid
# ax.plot(x0, y0, 'ro', label='Centroid')
ax[0].text(x0+0.05, y0+0.05, f'$(0,0)$')

# Plot axes
for k in [-1,0,1]:
    ax[0].axhline(y=y0+k*delta, color='gray', linestyle='--', linewidth=1)
    ax[0].axvline(x=x0+k*delta, color='gray', linestyle='--', linewidth=1)

# Annotate axis lines
# ax.text(x0 + 0.05, y0 + delta + 0.1, r'$x = x_0$', color='gray')
# ax.text(x0 + delta + 0.1, y0 + 0.05, r'$y = y_0$', color='gray')

# Define lines (edges of the diamond)
lines = [
    ((x0 - delta, y0), (x0, y0 + delta)),
    ((x0, y0 + delta), (x0 + delta, y0)),
    ((x0 + delta, y0), (x0, y0 - delta)),
    ((x0, y0 - delta), (x0 - delta, y0))
]

offset = 0.3  # outward distance from center for label placement

for (x1, y1), (x2, y2) in lines:
    # Midpoint
    x_mid = (x1 + x2) / 2
    y_mid = (y1 + y2) / 2

    # Direction vector of the edge
    dx = x2 - x1
    dy = y2 - y1

    # Normal vector pointing outward from (x0, y0)
    nx = y_mid - y0
    ny = -(x_mid - x0)
    norm = np.hypot(nx, ny)
    nx /= norm
    ny /= norm

    # Offset position away from center
    x_text = x_mid #+ offset * nx
    y_text = y_mid + 0.05 #+ offset * ny

    # Compute slope and angle
    m = (y2 - y1) / (x2 - x1)
    angle = np.degrees(np.arctan2(dy, dx))

    # Ensure text is upright
    if angle > 90:
        angle -= 180
    elif angle < -90:
        angle += 180

    # Place label
    ax[0].text(
        x_text, y_text,
        rf"$y = {'-' if m<0 else ''}x {'+' if (x_mid-x0)*m<0 else '-'}\rho$",
        fontsize=9,
        rotation=angle,
        rotation_mode='anchor',
        ha='center',
        va='bottom',
        bbox=dict(boxstyle='round,pad=0.5', fc='none', ec='none', alpha=0.7)
    )

# Symbolic tick labels
xticks = [x0 - delta, x0, x0 + delta]
yticks = [y0 - delta, y0, y0 + delta]
ax[0].set_xticks(xticks)
ax[0].set_yticks(yticks)
ax[0].set_xticklabels([r'$- \rho$', r'0', r'$+ \rho$'])
ax[0].set_yticklabels([r'$- \rho$', r'0', r'$+ \rho$'])

# Final formatting
ax[0].set_aspect('equal')
ratio_lim = 1.0
ax[0].set_xlim(x0 - ratio_lim * delta, x0 + ratio_lim * delta)
ax[0].set_ylim(y0 - ratio_lim * delta, y0 + ratio_lim * delta)
# ax.legend()
ax[0].set_title(r'$R_{\rho}$ domain')

# alternative plot for |E|

# Create a square grid
x = np.linspace(-1, 1, 1000)
y = np.linspace(-1, 1, 1000)
x, y = np.meshgrid(x, y)

z = x * y

mask_idx = (x + y <= 1) & (x + y >= -1) & (y - x <= 1) & (y - x >= -1)

z[~mask_idx] = np.nan


# Plot

c = ax[1].pcolormesh(x, y, abs(z), cmap='viridis', shading='auto')
ax[1].set_aspect('equal')
# Add colorbar
cb_ax = fig.add_subplot(gs[2])
cb = fig.colorbar(c, cax=cb_ax)

# cb = fig.colorbar(c, ax=ax[1], pad=0.1)

# Define diamond corners
diamond = np.array([
    [x0,         y0 + delta],
    [x0 + delta, y0        ],
    [x0,         y0 - delta],
    [x0 - delta, y0        ],
    [x0,         y0 + delta]
])


# Plot diamond
ax[1].plot(diamond[:, 0], diamond[:, 1], 0, 'black', linewidth=0.3)

# Custom x and y ticks
ax[1].set_xticks([x0 - delta, x0, x0 + delta])
ax[1].set_xticklabels([r'$- \rho$', r'$0$', r'$+ \rho$'])

ax[1].set_yticks([y0 - delta, y0, y0 + delta])
ax[1].set_yticklabels([r'$- \rho$', r'$0$', r'$+ \rho$'])

ax[1].set_xlabel('x')
# ax[1].set_ylabel('y')

# Set abstract tick labels based on \delta
cb.set_ticks([0, 0.25])
cb.set_ticklabels([r'$0$', r'$\rho^2/4$'])

ax[1].set_title(r'Value of $|E_{\rho}(x,y)|$')

# plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,6),dpi=250)
ax = fig.add_subplot(projection='3d')


x, y = np.mgrid[0:1:1000j, 0:1:1000j]
z = x * y

surf = ax.plot_surface(x, y, z, cmap='viridis', alpha=1.0,
                        antialiased=False, linewidth=0, edgecolor='none',)


# Set view angle (elevation and azimuth)
ax.view_init(elev=15, azim=-15)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')


plt.tight_layout()
# plt.show()

plt.savefig('test.png')

In [ ]:
fig = plt.figure(figsize=(10,4),dpi=250)
ax = fig.add_subplot(projection='3d')


x, y = np.mgrid[-1:1:1000j, -1:1:1000j]
z = x * y

mask_idx = (x + y <= 1) & (x + y >= -1) & (y - x <= 1) & (y - x >= -1)

z[~mask_idx] = np.nan

z_0 = x*0.
z_0[~mask_idx] = np.nan

# Parameters
x0, y0 = 0, 0
delta = 1


# Define diamond corners
diamond = np.array([
    [x0,         y0 + delta],
    [x0 + delta, y0        ],
    [x0,         y0 - delta],
    [x0 - delta, y0        ],
    [x0,         y0 + delta]
])


# Plot diamond
ax.plot(diamond[:, 0], diamond[:, 1], 0, 'k-')

surf = ax.plot_surface(x, y, abs(z), cmap='viridis', alpha=1.0,
                        antialiased=False, linewidth=0, edgecolor='none',)

# ax.plot_surface(x, y, z_0, color='C1', alpha=0.5,
#                        antialiased=False, linewidth=0, edgecolor='none',)

# Set view angle (elevation and azimuth)
ax.view_init(elev=15, azim=-15)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

ax.set_zlim([0,1])


plt.tight_layout()
# plt.show()

plt.savefig('test.png')

In [ ]:
# Create a square grid
x = np.linspace(-1, 1, 1000)
y = np.linspace(-1, 1, 1000)
x, y = np.meshgrid(x, y)

z = x * y

mask_idx = (x + y <= 1) & (x + y >= -1) & (y - x <= 1) & (y - x >= -1)

z[~mask_idx] = np.nan


# Plot
fig, ax = plt.subplots(figsize=(10,5),dpi=150)
c = ax.pcolormesh(x, y, abs(z), cmap='viridis', shading='auto')
ax.set_aspect('equal')
# Add colorbar
cb = fig.colorbar(c, ax=ax)

# Define diamond corners
diamond = np.array([
    [x0,         y0 + delta],
    [x0 + delta, y0        ],
    [x0,         y0 - delta],
    [x0 - delta, y0        ],
    [x0,         y0 + delta]
])


# Plot diamond
ax.plot(diamond[:, 0], diamond[:, 1], 0, 'black', linewidth=0.3)

# Custom x and y ticks
ax.set_xticks([x0 - delta, x0, x0 + delta])
ax.set_xticklabels([r'$x_0 - \delta$', r'$x_0$', r'$x_0 + \delta$'])

ax.set_yticks([y0 - delta, y0, y0 + delta])
ax.set_yticklabels([r'$y_0 - \delta$', r'$y_0$', r'$y_0 + \delta$'])

ax.set_xlabel('x')
ax.set_ylabel('y')

# Set abstract tick labels based on \delta
cb.set_ticks([0, 0.25])
cb.set_ticklabels([r'$0$', r'$\delta^2/4$'])
plt.show()

In [ ]:
N=5
faces_T = list_faces_from_N(N, method='triangles')
faces_P = list_faces_from_N(N, method='polygons')

# plot_faces_3d(faces_P)

faces_plus, faces_minus = list_faces_from_N_DC(N)

point = (0.3,0.4)
is_in_polygon = [Path(f[:,:2]).contains_point(point) for f in faces_P]
selected_polygon = faces_P[np.where(is_in_polygon)[0][0]]
is_triangle_in_polygon = [Path(selected_polygon[:,:2]).contains_point(tuple(f[:,:2].mean(axis=0))) for f in faces_T]


fig, ax = plt.subplots(1,2,figsize=(10,5),dpi=150)

ax[0].set_xlim([0,1])
ax[0].set_ylim([0,1])
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')
j=0
k=0
ax[0].plot([-1,2],[j/N+1,j/N-2],'k')
ax[0].plot([-1,2],[-1-1+j/N,2-1+j/N],'k--')
ax[0].add_collection(PolyCollection([selected_polygon[:,:2]], facecolors='C2', alpha=0.6))
for j in range(2*N):
    ax[0].plot([-1,2],[j/N+1,j/N-2],'k')
    ax[0].plot([-1,2],[-1-1+j/N,2-1+j/N],'k--')
ax[0].set_aspect('equal', adjustable='box')
ax[0].set_title('"DC" and "polygon" representation of $g_n$')
ax[0].legend(['Boundary between linear domains of $g_n^+$',
                'Boundary between linear domains of $g_n^-$',
                'A linear domain of $g_n$'])


ax[1].set_xlim([0,1])
ax[1].set_ylim([0,1])
ax[1].set_xlabel('x')
ax[1].set_ylabel('y')
faceCollection = PolyCollection([face[:,:2] for face in faces_T], facecolors='#FF000000', edgecolors='k')
ax[1].add_collection(PolyCollection([selected_polygon[:,:2]], facecolors='C2', alpha=0.6))
ax[1].add_collection(faceCollection)
ax[1].set_aspect('equal', adjustable='box')
ax[1].set_title('"triangle" representation of $g_n$')
ax[1].legend(['Domain of a pair of co-planar pieces'])

fig.tight_layout()

In [ ]:
fig = plt.figure(figsize=(10,4),dpi=150)
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
faceCollection = Poly3DCollection(faces_P,shade=False,facecolors='C2',edgecolors='k',alpha=0.5)
ax1.add_collection3d(faceCollection)
ax1.set_title('$g_n$')

ax2 = fig.add_subplot(1, 2, 2, projection='3d')
faceCollection = Poly3DCollection(faces_plus,shade=False,facecolors='C1',edgecolors='k',alpha=0.5)
ax2.add_collection3d(faceCollection)
faceCollection = Poly3DCollection(faces_minus,shade=False,facecolors='C0',edgecolors='k',alpha=0.5)
ax2.add_collection3d(faceCollection)
ax2.set_title('$g_n^+$ and $g_n^-$')
ax2.legend(['$g_n^+$','$g_n^-$'])

for ax in [ax1,ax2]:
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.view_init(elev=30, azim=-45)

fig.tight_layout()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = np.linspace(0, 1, 50) # 100 points between 0 and 1 for the x-axis
y = np.linspace(0, 1, 50) # 100 points between 0 and 1 for the y-axis
X, Y = np.meshgrid(x, y)
Z = X * Y

ax.plot_surface(X, Y, Z, cmap='viridis')

# 6. Set labels and title for clarity
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('z = xy')


target_error = 0.01
N = int(np.ceil(1/(4*np.sqrt(target_error))))

N = 8
eps = 1e-3/N

faces = []
for j in range(2*N):
    for k in range(2*N):
        x = (j-k)/(2*N) + 0.5
        y = (j+k+1)/(2*N) - 0.5
        if (x>=0) and (x<=1) and (y>=0) and (y<=1):
            pts = []
            for delta in [(-1,0),(0,-1),(1,0),(0,1)]:
                    xx = x + delta[0]/(2*N)
                    yy = y + delta[-1]/(2*N)
                    if (xx>=0-eps) and (xx<=1+eps) and (yy>=0-eps) and (yy<=1+eps):
                        pts.append([xx,yy,xx*yy])
            pts = np.array(pts)
            faces.append(pts)
            
faces_triang = []
for face in faces:
    if len(face)==3:
        faces_triang.append(face)
    else:
        x,y = face.mean(axis=0)[0:2]
        if (y>=x - eps) ^ (x+y>=1-eps):
            face = face[face[:,0].argsort(),:]
        else:
            face = face[face[:,1].argsort(),:]
        faces_triang.append(face[0:3,:])
        faces_triang.append(face[1:4,:])
        

x_plus = []
y_plus = []
xy_plus = []
x_minus = []
y_minus = []
xy_minus = []

for j in range(2*N+1):
    tmp_x = []
    tmp_y = []
    tmp_xy = []
    for k in range(2*N+1):
        x = (j-k)/(2*N) + 0.5
        y = (j+k)/(2*N) - 0.5
        if (x>=0-eps) and (x<=1+eps) and (y>=0-eps) and (y<=1+eps):
            tmp_x.append(x)
            tmp_y.append(y)
            tmp_xy.append(x*y)
    x_plus.append(tmp_x)
    y_plus.append(tmp_y)
    xy_plus.append(tmp_xy)
    
for k in range(2*N+1):
    tmp_x = []
    tmp_y = []
    tmp_xy = []
    for j in range(2*N+1):
        x = (j-k)/(2*N) + 0.5
        y = (j+k)/(2*N) - 0.5
        if (x>=0-eps) and (x<=1+eps) and (y>=0-eps) and (y<=1+eps):
            tmp_x.append(x)
            tmp_y.append(y)
            tmp_xy.append(x*y)
    x_minus.append(tmp_x)
    y_minus.append(tmp_y)
    xy_minus.append(tmp_xy)


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d',computed_zorder=False)
faceCollection = Poly3DCollection(faces_triang,shade=False,facecolors='C1',edgecolors='k',alpha=0.5)
ax.add_collection3d(faceCollection)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title(f'N = {N}, {len(faces_triang)} pieces, error = {100/(4*N)**2:.3f}%')
        

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d',computed_zorder=False)
faceCollection = Poly3DCollection(faces,shade=False,facecolors='C1',edgecolors='k',alpha=0.5)
ax.add_collection3d(faceCollection)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title(f'N = {N}, {len(faces)} pieces, error = {100/(4*N)**2:.3f}%')


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d',computed_zorder=False)
faceCollection = Poly3DCollection(faces,shade=False,facecolors='C1',alpha=0.5)
for n in range(len(x_plus)):
    ax.plot(x_plus[n],y_plus[n],xy_plus[n],'C2')
for n in range(len(x_minus)):
    ax.plot(x_minus[n],y_minus[n],xy_minus[n],'C3')
ax.plot([0,1,1,0,0],[0,0,1,1,0],[0,0,1,0,0],'k')
ax.add_collection3d(faceCollection)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title(f'N = {N}, {2*N} + {2*N} regions, error = {100/(4*N)**2:.3f}%')

print(f'Relative error: {100/(4*N)**2:.3f}%')

In [ ]:
# Custom formatter to remove trailing ".0"
class NoDecimalScalarFormatter(ScalarFormatter):
    def _set_format(self):
        self.format = "%d"  # integer formatting only

# n from 1 to 32
# n = np.arange(1, 33)
n = 2**np.arange(6)

# Functions from the table (linear constraints)
triangle = 16*n**2 + 4
polygon = 10*n*(n+1) + 4
dc = 14*n + 9
triangle_log = 16*n**2 + 2*np.ceil(2*np.log2(2*n)) + 4
polygon_log = 10*n*(n+1) + 2*np.ceil(np.log2(2*n*(n+1))) + 4
dc_log = 14*n + 4*np.ceil(np.log2(2*n)) + 9

# Continuous variables
triangle_cont = 12*n**2 + 1
polygon_cont = 6*n*(n+1) + 1
dc_cont = 12*n + 3
triangle_log_cont = 16*n**2 + 1
polygon_log_cont = 8*n*(n+1) + 1
dc_log_cont = 16*n + 3

# Binary variables
triangle_bin = 4*n**2
polygon_bin = 2*n*(n+1)
dc_bin = 4*n
triangle_log_bin = np.ceil(2*np.log2(2*n))
polygon_log_bin = np.ceil(np.log2(2*n*(n+1)))
dc_log_bin = 2*np.ceil(np.log2(2*n))


# Approximation error
error = 1/(16*n**2)

# Create the plot
fig, ax1 = plt.subplots(1,3,figsize=(10,4),dpi=150)
plt.subplots_adjust(wspace=0.0)

# # Primary y-axis: linear constraints
# ax1.plot(n, triangle, 'o-', label='Triangle')
# ax1.plot(n, polygon, 's-', label='Polygon')
# ax1.plot(n, dc, '^-', label='DC')
# ax1.plot(n, triangle_log, 'o--', label='Triangle (LogEnc)')
# ax1.plot(n, polygon_log, 's--', label='Polygon (LogEnc)')
# ax1.plot(n, dc_log, '^--', label='DC (LogEnc)')

# Primary y-axis: linear constraints
l1, = ax1[0].plot(n, triangle, '--', color='C0', label='Triangle')
l2, = ax1[0].plot(n, polygon, '--', color='C1', label='Polygon')
l3, = ax1[0].plot(n, dc, '--', color='C2', label='DC')
l4, = ax1[0].plot(n, triangle_log, '-', color='C0', label='Triangle (LogEnc)')
l5, = ax1[0].plot(n, polygon_log, '-', color='C1', label='Polygon (LogEnc)')
l6, = ax1[0].plot(n, dc_log, '-', color='C2', label='DC (LogEnc)')

ax1[0].set_xscale('log', base=2)
ax1[0].set_yscale('log')
ax1[0].set_xlabel('n')
ax1[0].set_ylabel('Number of constraints, variables')
ax1[0].set_ylim([1,10**5])

ax1[0].set_title('Linear constraints')

# Apply the integer-only formatter
formatter = NoDecimalScalarFormatter()
formatter.set_scientific(False)
ax1[0].xaxis.set_major_formatter(formatter)
# ax1[0].yaxis.set_major_locator(LogLocator(base=10.0, subs=None, numticks=10))
ax1[0].yaxis.set_minor_locator(LogLocator(base=10.0, subs=[], numticks=1))
# ax1[0].yaxis.set_major_formatter(formatter)


# Secondary y-axis: approximation error
ax2 = ax1[0].twinx()
l7, = ax2.plot(n, error, 'C3', label='Error')
ax2.set_yscale('log')
# ax2.set_ylabel('Maximum approximation error')
ax2.set_ylim([10**(-5),10**(0)])
ax2.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[], numticks=1))
# ax2.yaxis.set_major_formatter(formatter)
ax2.set_yticklabels([])

# ax1[0].legend(loc='upper center')
ax1[0].grid(True, which="both", ls="--", alpha=0.5)


# ax2.legend(loc='lower left')


## second figure


# Primary y-axis: linear constraints
ax1[1].plot(n, triangle_bin, '--', color='C0', label='Triangle')
ax1[1].plot(n, polygon_bin, '--', color='C1', label='Polygon')
ax1[1].plot(n, dc_bin, '--', color='C2', label='DC')
ax1[1].plot(n, triangle_log_bin, '-', color='C0', label='Triangle (LogEnc)')
ax1[1].plot(n, polygon_log_bin, '-', color='C1', label='Polygon (LogEnc)')
ax1[1].plot(n, dc_log_bin, '-', color='C2', label='DC (LogEnc)')

ax1[1].set_xscale('log', base=2)
ax1[1].set_yscale('log')
ax1[1].set_xlabel('n')
# ax1[1].set_ylabel('Linear Constraints')
ax1[1].set_ylim([1,10**5])

ax1[1].set_title('Binary variables')

# Apply the integer-only formatter
formatter = NoDecimalScalarFormatter()
formatter.set_scientific(False)
ax1[1].xaxis.set_major_formatter(formatter)
ax1[1].yaxis.set_minor_locator(LogLocator(base=10.0, subs=[], numticks=1))
# ax1.yaxis.set_major_formatter(formatter)
ax1[1].set_yticklabels([])

# Secondary y-axis: approximation error
ax2 = ax1[1].twinx()
ax2.plot(n, error, 'C3', label='Error')
ax2.set_yscale('log')
# ax2.set_ylabel('Maximum approximation error')
ax2.set_ylim([10**(-5),10**(0)])
ax2.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[], numticks=1))
# ax2.yaxis.set_major_formatter(formatter)
ax2.set_yticklabels([])

# ax1[1].legend(loc='upper center')
ax1[1].grid(True, which="both", ls="--", alpha=0.5)


# ax2.legend(loc='lower left')

## third figure

# Primary y-axis: linear constraints
ax1[2].plot(n, triangle_cont, '--', color='C0', label='Triangle')
ax1[2].plot(n, polygon_cont, '--', color='C1', label='Polygon')
ax1[2].plot(n, dc_cont, '--', color='C2', label='DC')
ax1[2].plot(n, triangle_log_cont, '-', color='C0', label='Triangle (LogEnc)')
ax1[2].plot(n, polygon_log_cont, '-', color='C1', label='Polygon (LogEnc)')
ax1[2].plot(n, dc_log_cont, '-', color='C2', label='DC (LogEnc)')

ax1[2].set_xscale('log', base=2)
ax1[2].set_yscale('log')
ax1[2].set_xlabel('n')
# ax1[1].set_ylabel('Linear Constraints')
ax1[2].set_ylim([1,10**5])
ax1[2].set_yticklabels([])

ax1[2].set_title('Continuous variables')

# Apply the integer-only formatter
formatter = NoDecimalScalarFormatter()
formatter.set_scientific(False)
ax1[2].xaxis.set_major_formatter(formatter)
ax1[2].yaxis.set_minor_locator(LogLocator(base=10.0, subs=[], numticks=1))
# ax1.yaxis.set_major_formatter(formatter)

# Secondary y-axis: approximation error
ax2 = ax1[2].twinx()
ax2.plot(n, error, 'C3', label='Error')
ax2.set_yscale('log')
ax2.set_ylabel('Maximum approximation error')
ax2.set_ylim([10**(-5),10**(0)])
ax2.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[], numticks=1))
# ax2.yaxis.set_major_formatter(formatter)

# ax1[1].legend(loc='upper center')
ax1[2].grid(True, which="both", ls="--", alpha=0.5)


# ax2.legend(loc='lower left')

# Make legend belong to the figure
fig.legend(
    handles=[l1,l2,l3,l4,l5,l6,l7],  # first 9 entries for 3x3
    loc='lower center',
    bbox_to_anchor=(0.5, +0.05),
    fontsize=9,
    ncol=7
)

plt.subplots_adjust(bottom=0.28)

# plt.title('Linear Constraints vs n (Log-Log) with Approximation Error')
# plt.tight_layout()
plt.show()